<a href="https://colab.research.google.com/github/HHH023/coastline_extraction/blob/main/ee_classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Supervised classification with GEE**

Classification and Regression Trees (CART)


In [ ]:
# Import earthengine API
import ee
# Authenticate and initialise 
ee.Authenticate()
ee.Initialize()

In [2]:
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)
  
rec = ee.Geometry.Rectangle([151.89, -32.84, 152.13, -32.74])

# Map the function over one year of data and take the median.
# Load Sentinel-2 TOA reflectance data
# Pre-filter to get less cloudy granules.
dataset = (ee.ImageCollection('COPERNICUS/S2')
       .filterBounds(rec)
       .filterDate('2019-01-01', '2019-12-30')  
       .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
       .map(maskS2clouds))

In [3]:
# Make a cloud-free Landsat 8 TOA composite (from raw imagery)
image = dataset.first()


bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B8A','B9', 'B10', 'B11', 'B12'];

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('users/jingyuhu98/traininggeo')

# This property stores the land cover labels as consecutive
# integers starting from zero.
label = 'class'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(points,properties=[label],scale=10)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)


In [4]:
import folium
def addLayer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.addLayer = addLayer

In [5]:
# Plot the result

import folium
!pip install geehydro # Life saver for plotting GEE stuff with Python!
import geehydro
# Use folium to visualize the imagery.
map = folium.Map(location=[-32.8, 152],zoom_start=10)

map.addLayer(image, {'bands': ['B4', 'B3', 'B2']}, 'image')
map.addLayer(classified, {'min':0, 'max':3, 'palette': ['blue', 'black', 'green', 'yellow']}, 'classification')
folium.LayerControl().add_to(map)
map


  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10141 sha256=ab7ab37b5b56674a0987c2f254c2bd8f8f0f1561d79f0d256582d35f84b82022
  Stored in directory: /root/.cache/pip/wheels/49/6c/92/019aeb37663078e6666afa9dd336e2d8df72d5683d7d357697
Successfully built geehydro
